In [85]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/ubuntu/varios/skforecast


In [86]:
from skforecast.preprocessing import series_long_to_dict, exog_long_to_dict
import pandas as pd
import numpy as np

In [87]:
series_1 = pd.Series(np.random.normal(0, 1, 10), index = pd.date_range(start='1-1-2000', periods=10, freq='D'), name = 'series_1')
series_2 = pd.Series(np.random.normal(0, 1, 10), index = pd.date_range(start='1-1-2000', periods=10, freq='D'), name = 'series_2')
series_long = pd.concat([series_1, series_2], axis = 1)
series_long = series_long.melt(var_name = 'id', value_name = 'value', ignore_index = False).reset_index().rename(columns = {'index': 'datetime'})
series_long

,datetime,id,value
0,2000-01-01,series_1,-0.453573
1,2000-01-02,series_1,0.577036
2,2000-01-03,series_1,-0.179167
3,2000-01-04,series_1,-0.817858
4,2000-01-05,series_1,0.853728
5,2000-01-06,series_1,0.601390
6,2000-01-07,series_1,0.305726
7,2000-01-08,series_1,0.225149
8,2000-01-09,series_1,0.377583
9,2000-01-10,series_1,-0.760975


In [88]:
exog_series_1 = pd.DataFrame({
    'exog_1': np.random.normal(0, 1, 10),
    'exog_2': np.random.choice(["A", "B", "C"], 10),
    'exog_3': np.random.randint(0, 10, 10),
    'series': 'series_1',
}, index = pd.date_range(start='1-1-2000', periods=10, freq='D'))
exog_series_2 = pd.DataFrame({
    'exog_1': np.random.normal(0, 1, 10),
    'exog_2': np.random.choice(["A", "B"], 10),
    'exog_3': np.random.randint(0, 10, 10),
    'series': 'series_2',
}, index = pd.date_range(start='1-1-2000', periods=10, freq='D'))
exog_long = pd.concat([exog_series_1, exog_series_2], axis = 0).reset_index().rename(columns = {'index': 'datetime'})
exog_long['exog_2'] = exog_long['exog_2'].astype('category')
exog_long['series'] = exog_long['series'].astype('category')
exog_long

,datetime,exog_1,exog_2,exog_3,series
0,2000-01-01,-0.505396,B,6,series_1
1,2000-01-02,-0.478147,B,9,series_1
2,2000-01-03,-0.940430,B,8,series_1
3,2000-01-04,-1.345530,A,5,series_1
4,2000-01-05,-2.260707,A,3,series_1
5,2000-01-06,-0.387024,C,2,series_1
6,2000-01-07,-0.778508,A,8,series_1
7,2000-01-08,1.244051,C,0,series_1
8,2000-01-09,0.204480,B,9,series_1
9,2000-01-10,0.151021,B,4,series_1


In [89]:
series_long_to_dict(
    data = series_long,
    series_id='id',
    index = 'datetime',
    values = 'value',
    freq='D',
)

{'series_1': 2000-01-01   -0.453573
 2000-01-02    0.577036
 2000-01-03   -0.179167
 2000-01-04   -0.817858
 2000-01-05    0.853728
 2000-01-06    0.601390
 2000-01-07    0.305726
 2000-01-08    0.225149
 2000-01-09    0.377583
 2000-01-10   -0.760975
 Freq: D, Name: series_1, dtype: float64,
 'series_2': 2000-01-01   -0.020115
 2000-01-02   -0.570302
 2000-01-03   -1.858330
 2000-01-04   -0.355471
 2000-01-05    0.858392
 2000-01-06    0.122227
 2000-01-07   -0.951924
 2000-01-08    0.501109
 2000-01-09    0.966342
 2000-01-10   -1.221275
 Freq: D, Name: series_2, dtype: float64}

In [90]:
exog_long_to_dict(
    data = exog_long,
    series_id='series',
    index = 'datetime',
    freq='D',
)

{'series_1':               exog_1 exog_2  exog_3
 2000-01-01 -0.505396      B       6
 2000-01-02 -0.478147      B       9
 2000-01-03 -0.940430      B       8
 2000-01-04 -1.345530      A       5
 2000-01-05 -2.260707      A       3
 2000-01-06 -0.387024      C       2
 2000-01-07 -0.778508      A       8
 2000-01-08  1.244051      C       0
 2000-01-09  0.204480      B       9
 2000-01-10  0.151021      B       4,
 'series_2':               exog_1 exog_2  exog_3
 2000-01-01  1.311253      A       4
 2000-01-02  1.381355      A       1
 2000-01-03 -0.459153      B       6
 2000-01-04 -0.114396      A       9
 2000-01-05 -1.572003      B       1
 2000-01-06  1.497852      A       8
 2000-01-07  1.257496      A       6
 2000-01-08 -0.901329      A       5
 2000-01-09 -0.365849      B       0
 2000-01-10  0.496952      A       3}

In [91]:
exog_long = exog_long.loc[[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19], :].copy()
exog_dict = exog_long_to_dict(
    data = exog_long,
    series_id='series',
    index = 'datetime',
    freq='D',
)

/home/ubuntu/varios/skforecast/skforecast/preprocessing/preprocessing.py:502: MissingValuesWarning: Exogenous variables for series 'series_1' are incomplete. NaNs have been introduced after setting the frequency. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=MissingValuesWarning)
  


In [92]:
display(exog_dict['series_1'].dtypes)
exog_dict['series_1']

exog_1     float64
exog_2    category
exog_3     float64
dtype: object

,exog_1,exog_2,exog_3
2000-01-01,-0.505396,B,6.0
2000-01-02,-0.478147,B,9.0
2000-01-03,-0.940430,B,8.0
2000-01-04,-1.345530,A,5.0
2000-01-05,-2.260707,A,3.0
2000-01-06,NaN,NaN,NaN
2000-01-07,-0.778508,A,8.0
2000-01-08,1.244051,C,0.0
2000-01-09,0.204480,B,9.0
2000-01-10,0.151021,B,4.0


In [93]:
display(exog_dict['series_2'].dtypes)
exog_dict['series_2']

exog_1     float64
exog_2    category
exog_3     float64
dtype: object

,exog_1,exog_2,exog_3
2000-01-01,1.311253,A,4.0
2000-01-02,1.381355,A,1.0
2000-01-03,-0.459153,B,6.0
2000-01-04,-0.114396,A,9.0
2000-01-05,-1.572003,B,1.0
2000-01-06,1.497852,A,8.0
2000-01-07,1.257496,A,6.0
2000-01-08,-0.901329,A,5.0
2000-01-09,-0.365849,B,0.0
2000-01-10,0.496952,A,3.0


In [84]:
exog_dict['series_2'].astype(dict(exog_dict['series_1'].dtypes))

,exog_1,exog_2,exog_3
2000-01-01,-0.042886,B,1.0
2000-01-02,-1.578853,B,1.0
2000-01-03,0.805355,B,0.0
2000-01-04,-0.631885,A,9.0
2000-01-05,-0.253022,B,1.0
2000-01-06,1.727674,A,5.0
2000-01-07,0.565419,B,9.0
2000-01-08,0.549961,A,8.0
2000-01-09,0.035389,B,6.0
2000-01-10,-0.626402,A,9.0


In [70]:
original_dtype = dict(exog_dict['series_1'].dtypes)
final_dtype = {k: None for k in original_dtype.keys()}
final_dtype.update({'exog_2': 'category'})
final_dtype

{'exog_1': None, 'exog_2': 'category', 'exog_3': None}

In [71]:
cols_float_dtype = set(['a', 'b'])
cols_float_dtype.update(['a', 'b'])
cols_float_dtype.update([])
cols_float_dtype.update(['a', 'c'])
cols_float_dtype.update(["a", "c"])
cols_float_dtype

{'a', 'b', 'c'}

In [72]:
exog_dict['series_1'].select_dtypes(include=int).columns

Index([], dtype='object')